# Lab 4: PolyHAR CNN

## Imports

In [ ]:
import copy
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils import to_categorical

## Load PolyHAR dataset (EllcieHAR format)

In [ ]:
x_full = []
y_full = []
with open('polyhar.csv') as f:
    next(f) # Skip header
    for l in f:
        d = l.split(';')
        x_full.append([float(d[1]), float(d[2]), float(d[3])]) # Store 3-axis accelerometer adata
        y_full.append(1 if 'Positive' in d[4] else 0) # Store positive labels

x_full = np.array(x_full)
y_full = np.array(y_full)

## Windowing

In [ ]:
SIZE = 32
CLASSES = 2
windowscount = np.ceil(x_full.shape[0]/SIZE).astype(int)
x_full = np.resize(x_full, (windowscount, SIZE, x_full.shape[-1]))
y_full = np.resize(y_full, (windowscount, SIZE))
y_full = np.array([np.bincount(w).argmax() for w in y_full]) # Select label with highest number of occurence for each window
y_full = to_categorical(y_full, num_classes=CLASSES) # Convert back to one-hot encoding

## Train/test split

In [ ]:
RATIO = 0.2 # 20% test, 80% train
n = int(len(x_full) * RATIO)

# Randomize windows
p = np.random.permutation(len(x_full))
x_full = x_full[p]
y_full = y_full[p]

x_test = x_full[-n:]
y_test = y_full[-n:]

x_train = x_full[:-len(x_test)]
y_train = y_full[:-len(y_test)]

## Export pre-processed dataset

In [ ]:
np.savetxt('x_train_polyhar.csv', x_train.reshape((x_train.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_train_polyhar.csv', y_train, delimiter=',', fmt='%s')
np.savetxt('x_test_polyhar.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_polyhar.csv', y_test, delimiter=',', fmt='%s')

## Build model

In [ ]:
model = Sequential()
model.add(Input(shape=(SIZE, 3)))
model.add(Conv1D(filters=2, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=CLASSES))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-5)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Train model

In [ ]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

## Evaluate model on test dataset

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

## Save trained model

In [ ]:
model.save('lab4_polyhar.h5')

## Remove SoftMax layer

In [ ]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [ ]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/e9490f8fdf8d.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

## Generate C code for the trained model with 16-bit fixed-point representation

In [ ]:
res = kerascnn2c.Converter(output_path=Path('polyhar_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('polyhar_model_fixed.h', 'w') as f:
    f.write(res)

## Compile the 16-bit fixed-point C code for x86 and evaluate

In [ ]:
!g++ -Wall -Wextra -pedantic -Ofast -o polyhar_fixed -Ipolyhar_output_fixed/ polyhar_output_fixed/model.c main.cpp 
!./polyhar_fixed x_test_polyhar.csv y_test_polyhar.csv